REF: https://towardsdatascience.com/how-to-improve-any-ml-dl-performance-by-10-easily-90dbbd01a4b3 

In [1]:
#install packages
!pip install nni 

     |████████████████████████████████| 56.1 MB 1.5 MB/s 
     |████████████████████████████████| 112 kB 35.3 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 130 kB 40.7 MB/s 
     |████████████████████████████████| 127 kB 10.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# MNIST Classification **without NNI**

In [2]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import nni

[2022-04-29 05:47:28] INFO (numexpr.utils/MainThread) NumExpr defaulting to 2 threads.


In [ ]:
def train(model, device, train_loader, loss_fn, optimiser):
  size = len(train_loader.dataset)
  model.train()
  
  for batch, (X,y) in enumerate(train_loader):
    X, y = X.to(device), y.to(device)

    #compute prediction error 
    pred = model(X)
    loss = loss_fn(pred,  y)

    #backpropagation 
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if batch %100 == 0:
        loss, current = loss.item(), batch * len(X)
        print (f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(model, device, loss_fn, test_loader):
  size = len(test_loader.dataset)
  num_batches = len(test_loader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X, y in test_loader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss/=num_batches
  correct /= size 
  print(f"Test Error: \n Accuracy {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")
  return correct

In [ ]:
def main(args):
  training_data = datasets.FashionMNIST(root="data", train=True, download=True,transform=ToTensor(),)
  testing_data = datasets.FashionMNIST(root="data", train=False, download=True,transform=ToTensor(),)

  train_dataloader = DataLoader(training_data,batch_size=args['batch_size'])
  test_dataloader = DataLoader(testing_data,batch_size = 64)

  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(f"Using {device} device")

  #Define Model 
  class NeuralNetwork(nn.Module):
    def __init__(self, hidden_size1, hidden_size2 ):
      super(NeuralNetwork,self).__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(28*28, hidden_size1),
          nn.ReLU(), 
          nn.Linear(hidden_size1, hidden_size2),
          nn.ReLU(),
          nn.Linear(hidden_size2,10)
      )
    def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits
  
  model = NeuralNetwork(hidden_size1=args['hidden_size1'],
                        hidden_size2=args['hidden_size2']).to(device)
  print(model)
  loss_fn = nn.CrossEntropyLoss()
  optimiser = torch.optim.SGD(model.parameters(), lr=args['lr'])

  for epoch in range(10):
    print(f'Epoch {epoch+1}\n ------------------------')
    train(model, device, train_dataloader, loss_fn, optimiser)
    test_acc = test(model, device, loss_fn, test_dataloader)
    print(test_acc)
  print(f'final accuracy:{test_acc}')

if __name__ == '__main__':
  params = {
      'batch_size':32,
      'hidden_size1':128,
      'hidden_size2':128,
      'lr':0.001,
      'momentum':0.5
  }
  main(params)




Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)
Epoch 1
 ------------------------
loss: 2.305076 [    0/60000]
loss: 2.299222 [ 3200/60000]
loss: 2.288804 [ 6400/60000]
loss: 2.280372 [ 9600/60000]
loss: 2.272156 [12800/60000]
loss: 2.264546 [16000/60000]
loss: 2.247690 [19200/60000]
loss: 2.249331 [22400/60000]
loss: 2.256461 [25600/60000]
loss: 2.209541 [28800/60000]
loss: 2.203690 [32000/60000]
loss: 2.188910 [35200/60000]
loss: 2.211001 [38400/60000]
loss: 2.166477 [41600/60000]
loss: 2.138067 [44800/60000]
loss: 2.115976 [48000/60000]
loss: 2.129153 [51200/60000]
loss: 2.091841 [54400/60000]
loss: 2.033167 [57600/60000]
Test Error: 
 Accuracy 54.7%, Avg Loss: 2.034706 

0.5473
Epoch 2
 ------------

# Hyperparam Search with NNI

## STEP1. search.py

In [3]:
from nni.experiment import Experiment
from pathlib import Path
import os, sys
import time

In [ ]:
#%% add parent_dir
# parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(parent_dir)

In [4]:
params = {
    'batch_size':32,
    'hidden_size1':128,
    'hidden_size2':128,
    'lr':0.001,
    'momentum':0.5
}

In [5]:
search_space = {
    'batch_size':{'_type':'choice', '_value':[32, 64, 128, 256]},
    'hidden_size1':{'_type':'choice', '_value':[64, 128, 256, 512]},
    'hidden_size2':{'_type':'choice', '_value':[64, 128, 256, 512]},
    'lr':{'_type':'loguniform', '_value':[0.0001, 0.1]},
    'momentum':{'_type':'uniform','_value':[0 , 1]}
}

In [ ]:
experiment = Experiment('local')
experiment.config.trial_code_directory = '.'
experiment.config.trial_command = 'python mnist_nni.py' # command to run the code 
experiment.config.search_space = search_space
experiment.config.tuner.name = 'TPE'
experiment.config.tuner.class_args['optimize_mode'] = 'maximize'
experiment.config.max_trial_number = 30                 # number of experiments to run. In general, TPE requires min 20 trials to warm up.
experiment.config.trial_gpu_number = 0                  # CUDA is required when it’s greater than zero.
experiment.config.trial_concurrency = 2
experiment.run(8082)

input ('Press Enter to Quit')
experiment.stop()

[2022-04-29 07:00:16] Creating experiment, Experiment ID: 2vyfw3mp
[2022-04-29 07:00:16] Starting web server...
[2022-04-29 07:00:17] Setting up...
[2022-04-29 07:00:17] Web portal URLs: http://127.0.0.1:8082 http://172.28.0.2:8082


In [ ]:
from google.colab import drive
drive.mount('/content/drive')